In [1]:
!pip install xlrd

In [22]:
import pandas as pd 
import xlrd
import numpy as np
from cStringIO import StringIO

# Prepare 'coefs' dataframe

In [3]:
df = pd.read_excel('combined_model_results.xlsx', sheet_name=1, header=0, index_col = None)
df.reset_index(level=0, inplace=True) # reset index

In [4]:
coefs = df[df.loc[:,('index')] == 'coef'] # get only rows that are coefficients

In [5]:
import warnings; warnings.simplefilter('ignore')

# add columns for missing (baseline) categories
coefs['age_15-19'] = 0
coefs['los_1-2'] = 0
coefs['occupation_fam_16'] = 0
coefs['superv_status_2'] = 0
print coefs.shape # There should be 54 variables + 1 index column + 1 agency column
coefs.head() 

(71, 56)


,index,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,age_50-54,age_55-59,age_60-64,...,occupation_fam_21,superv_status_4,superv_status_5,superv_status_6,superv_status_7,superv_status_8,age_15-19,los_1-2,occupation_fam_16,superv_status_2
0,coef,-0.787061,-0.825141,-0.872568,-0.942350,-1.093115,-1.378620,-1.203198,-0.526240,-0.008261,...,-7.684039,-6.264937,-6.640237,-7.028278,-0.073456,-0.074111,0,0,0,0
7,coef,NaN,-0.451929,0.084609,0.089965,-0.264928,0.145959,0.895556,0.645825,0.531235,...,NaN,NaN,NaN,NaN,-4.519053,-0.261383,0,0,0,0
14,coef,0.181545,0.265398,0.190734,-0.109149,-0.345021,-0.662591,-0.429656,0.286429,0.746830,...,0.092079,0.270890,0.056033,-8.599736,0.121974,-0.0617992,0,0,0,0
21,coef,0.003399,-0.146261,-0.379149,-0.438651,-0.625441,-0.781969,-0.515652,0.061743,0.477809,...,0.499116,0.128709,0.038346,-0.627838,0.276514,-0.127088,0,0,0,0
28,coef,-0.152375,-0.151193,-0.189924,-0.359776,-0.630608,-0.795759,-0.456254,0.289879,0.891721,...,-0.632342,-0.105356,0.013786,-0.235458,-0.086036,0.0811195,0,0,0,0


In [6]:
# reorder headers to match ordering of normalized data
print list(coefs.columns.values) # get list of column names

coefs = coefs[['agency', 
         'age_15-19', 'age_20-24', 'age_25-29', 'age_30-34', 'age_35-39', 'age_40-44', 
         'age_45-49', 'age_50-54', 'age_55-59', 'age_60-64', 'age_65-69', 'age_70-74', 
         'age_75+', 'age_UNSP', 
         'los_< 1', 'los_1-2', 'los_3-4', 'los_5-9', 'los_10-14', 'los_15-19', 
         'los_20-24', 'los_25-29',  'los_30-34', 'los_35+', 'los_UNSP', 
         'occupation_fam_01', 'occupation_fam_02', 'occupation_fam_03', 'occupation_fam_04', 
         'occupation_fam_05', 'occupation_fam_06', 'occupation_fam_07', 'occupation_fam_08', 
         'occupation_fam_09', 'occupation_fam_10', 'occupation_fam_11', 'occupation_fam_12', 
         'occupation_fam_13', 'occupation_fam_14', 'occupation_fam_15', 'occupation_fam_16', 
         'occupation_fam_17', 'occupation_fam_18', 'occupation_fam_19', 'occupation_fam_20', 
         'occupation_fam_21', 
         'superv_status_2', 'superv_status_4', 'superv_status_5', 'superv_status_6', 
         'superv_status_7', 'superv_status_8',
         'basic_pay', 'basic_pay_squared']]

['index', u'age_20-24', u'age_25-29', u'age_30-34', u'age_35-39', u'age_40-44', u'age_45-49', u'age_50-54', u'age_55-59', u'age_60-64', u'age_65-69', u'age_70-74', u'age_75+', u'age_UNSP', u'agency', u'basic_pay', u'basic_pay_squared', u'los_10-14', u'los_15-19', u'los_20-24', u'los_25-29', u'los_3-4', u'los_30-34', u'los_35+', u'los_5-9', u'los_< 1', u'los_UNSP', u'occupation_fam_01', u'occupation_fam_02', u'occupation_fam_03', u'occupation_fam_04', u'occupation_fam_05', u'occupation_fam_06', u'occupation_fam_07', u'occupation_fam_08', u'occupation_fam_09', u'occupation_fam_10', u'occupation_fam_11', u'occupation_fam_12', u'occupation_fam_13', u'occupation_fam_14', u'occupation_fam_15', u'occupation_fam_17', u'occupation_fam_18', u'occupation_fam_19', u'occupation_fam_20', u'occupation_fam_21', u'superv_status_4', u'superv_status_5', u'superv_status_6', u'superv_status_7', u'superv_status_8', 'age_15-19', 'los_1-2', 'occupation_fam_16', 'superv_status_2']


In [7]:
print coefs.shape # should be 54 variables + 1 agency
print coefs.columns.values
coefs.head()

(71, 55)
[u'agency' 'age_15-19' u'age_20-24' u'age_25-29' u'age_30-34' u'age_35-39'
 u'age_40-44' u'age_45-49' u'age_50-54' u'age_55-59' u'age_60-64'
 u'age_65-69' u'age_70-74' u'age_75+' u'age_UNSP' u'los_< 1' 'los_1-2'
 u'los_3-4' u'los_5-9' u'los_10-14' u'los_15-19' u'los_20-24' u'los_25-29'
 u'los_30-34' u'los_35+' u'los_UNSP' u'occupation_fam_01'
 u'occupation_fam_02' u'occupation_fam_03' u'occupation_fam_04'
 u'occupation_fam_05' u'occupation_fam_06' u'occupation_fam_07'
 u'occupation_fam_08' u'occupation_fam_09' u'occupation_fam_10'
 u'occupation_fam_11' u'occupation_fam_12' u'occupation_fam_13'
 u'occupation_fam_14' u'occupation_fam_15' 'occupation_fam_16'
 u'occupation_fam_17' u'occupation_fam_18' u'occupation_fam_19'
 u'occupation_fam_20' u'occupation_fam_21' 'superv_status_2'
 u'superv_status_4' u'superv_status_5' u'superv_status_6'
 u'superv_status_7' u'superv_status_8' u'basic_pay' u'basic_pay_squared']


,agency,age_15-19,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,age_50-54,age_55-59,...,occupation_fam_20,occupation_fam_21,superv_status_2,superv_status_4,superv_status_5,superv_status_6,superv_status_7,superv_status_8,basic_pay,basic_pay_squared
0,NU,0,-0.787061,-0.825141,-0.872568,-0.942350,-1.093115,-1.378620,-1.203198,-0.526240,...,-0.072940,-7.684039,0,-6.264937,-6.640237,-7.028278,-0.073456,-0.074111,-0.023504,0.000070
7,FW,0,NaN,-0.451929,0.084609,0.089965,-0.264928,0.145959,0.895556,0.645825,...,NaN,NaN,0,NaN,NaN,NaN,-4.519053,-0.261383,-0.026951,0.000104
14,TD,0,0.181545,0.265398,0.190734,-0.109149,-0.345021,-0.662591,-0.429656,0.286429,...,0.417590,0.092079,0,0.270890,0.056033,-8.599736,0.121974,-0.0617992,-0.009408,0.000035
21,CM,0,0.003399,-0.146261,-0.379149,-0.438651,-0.625441,-0.781969,-0.515652,0.061743,...,-0.091165,0.499116,0,0.128709,0.038346,-0.627838,0.276514,-0.127088,-0.005157,-0.000002
28,TR,0,-0.152375,-0.151193,-0.189924,-0.359776,-0.630608,-0.795759,-0.456254,0.289879,...,-0.610964,-0.632342,0,-0.105356,0.013786,-0.235458,-0.086036,0.0811195,-0.008659,0.000033


In [8]:
# fill in NaN values with 0
coefs.fillna(0, inplace=True)
coefs.head()

,agency,age_15-19,age_20-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,age_50-54,age_55-59,...,occupation_fam_20,occupation_fam_21,superv_status_2,superv_status_4,superv_status_5,superv_status_6,superv_status_7,superv_status_8,basic_pay,basic_pay_squared
0,NU,0,-0.787061,-0.825141,-0.872568,-0.942350,-1.093115,-1.378620,-1.203198,-0.526240,...,-0.072940,-7.684039,0,-6.264937,-6.640237,-7.028278,-0.073456,-0.074111,-0.023504,0.000070
7,FW,0,0.000000,-0.451929,0.084609,0.089965,-0.264928,0.145959,0.895556,0.645825,...,0.000000,0.000000,0,0.000000,0.000000,0.000000,-4.519053,-0.261383,-0.026951,0.000104
14,TD,0,0.181545,0.265398,0.190734,-0.109149,-0.345021,-0.662591,-0.429656,0.286429,...,0.417590,0.092079,0,0.270890,0.056033,-8.599736,0.121974,-0.061799,-0.009408,0.000035
21,CM,0,0.003399,-0.146261,-0.379149,-0.438651,-0.625441,-0.781969,-0.515652,0.061743,...,-0.091165,0.499116,0,0.128709,0.038346,-0.627838,0.276514,-0.127088,-0.005157,-0.000002
28,TR,0,-0.152375,-0.151193,-0.189924,-0.359776,-0.630608,-0.795759,-0.456254,0.289879,...,-0.610964,-0.632342,0,-0.105356,0.013786,-0.235458,-0.086036,0.081120,-0.008659,0.000033


# Prepare norm_data and merge 'coefs' 

In [9]:
# read in normalized data as left table. Join coefs to normalized data on 'agency'
norm_data = pd.read_csv('norm_data.csv')
norm_data = norm_data.drop(labels='Unnamed: 0', axis=1) # drop index column
print 'Shape of normalized data frame: ', norm_data.shape

# merge dataframes
pred_haz = pd.merge(norm_data, coefs, how='left', left_on='agency_cat', right_on='agency')
print 'Shape of joined data frame: ', pred_haz.shape
print pred_haz.columns.values
pred_haz.head()

Shape of normalized data frame:  (769918, 59)
Shape of joined data frame:  (769918, 114)
['emp_id' 'agency_cat' 'occupation_cat' 'basic_pay_x' 'basic_pay_squared_x'
 'age_15_19' 'age_20_24' 'age_25_29' 'age_30_34' 'age_35_39' 'age_40_44'
 'age_45_49' 'age_50_54' 'age_55_59' 'age_60_64' 'age_65_69' 'age_70_74'
 'age_75' 'age_UNSP_x' 'los_1' 'los_1_2' 'los_3_4' 'los_5_9' 'los_10_14'
 'los_15_19' 'los_20_24' 'los_25_29' 'los_30_34' 'los_35' 'los_UNSP_x'
 'occupation_fam_01_x' 'occupation_fam_02_x' 'occupation_fam_03_x'
 'occupation_fam_04_x' 'occupation_fam_05_x' 'occupation_fam_06_x'
 'occupation_fam_07_x' 'occupation_fam_08_x' 'occupation_fam_09_x'
 'occupation_fam_10_x' 'occupation_fam_11_x' 'occupation_fam_12_x'
 'occupation_fam_13_x' 'occupation_fam_14_x' 'occupation_fam_15_x'
 'occupation_fam_16_x' 'occupation_fam_17_x' 'occupation_fam_18_x'
 'occupation_fam_19_x' 'occupation_fam_20_x' 'occupation_fam_21_x'
 'superv_status_2_x' 'superv_status_4_x' 'superv_status_5_x'
 'superv_status

,emp_id,agency_cat,occupation_cat,basic_pay_x,basic_pay_squared_x,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,...,occupation_fam_20_y,occupation_fam_21_y,superv_status_2_y,superv_status_4_y,superv_status_5_y,superv_status_6_y,superv_status_7_y,superv_status_8_y,basic_pay_y,basic_pay_squared_y
0,898655,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.01261,-0.017117,-0.000102
1,2257068,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.01261,-0.017117,-0.000102
2,1745142,AA,P,113346.0,1.284732e+10,0,0,0,1,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.01261,-0.017117,-0.000102
3,5050496,AA,P,75621.0,5.718536e+09,0,0,0,1,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.01261,-0.017117,-0.000102
4,8638776,AA,P,157100.0,2.468041e+10,0,0,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.01261,-0.017117,-0.000102


# Prepare cumulative baseline hazard and merge with pred_haz

In [10]:
# Load baseline hazards
df2 = pd.read_excel('combined_model_results.xlsx', sheetname=1, header=0, index_col = None)

# Replace NaNs with 0
df2.fillna(0, inplace=True)

time = list(df2.columns.values)[1:-1]
time

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [11]:
# Create cumulative hazards

base_cum_haz = df2[[0, 1, 'agency']]                        # build base_cum_haz on baseline hazard t0 and t1
base_cum_haz.columns = ['t0', 't1', 'agency_cat']           # rename columns
base_cum_haz = base_cum_haz[['agency_cat', 't0', 't1']]     # reoder so 'agency_cat' comes first
for i in time:                                              # iterate to create cumulative hazard based on baseline hazard (df2)
  base_cum_haz['t' + str(i)] = base_cum_haz['t' + str(i-1)] + df2[i]

base_cum_haz.head()

,agency_cat,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18
0,NU,0,0.007019,0.014567,0.021835,0.031083,0.038341,0.049278,0.055494,0.060073,0.069755,0.079527,0.094438,0.104153,0.117430,0.117430,0.117430,0.117430,0.117430,0.117430
1,FW,0,0.017693,0.036116,0.045553,0.065430,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128,0.108128
2,TD,0,0.008695,0.018453,0.028979,0.041710,0.052723,0.062583,0.073171,0.079022,0.094075,0.110007,0.124513,0.132466,0.137277,0.137277,0.153739,0.153739,0.153739,0.153739
3,CM,0,0.008776,0.018803,0.029091,0.040919,0.050887,0.060129,0.071113,0.082270,0.092336,0.106041,0.118207,0.123586,0.133905,0.147014,0.171237,0.171237,0.171237,0.171237
4,TR,0,0.008402,0.018308,0.028946,0.041566,0.052120,0.063988,0.075521,0.086962,0.095868,0.106583,0.116450,0.127173,0.135576,0.138415,0.156261,0.211067,0.211067,0.211067


In [12]:
# merge with pred_haz based on agency_cat
pred_haz = pd.merge(pred_haz, base_cum_haz, how='left', on='agency_cat')
print 'Shape of joined data frame: ', pred_haz.shape
print pred_haz.columns.values

Shape of joined data frame:  (769918, 133)
['emp_id' 'agency_cat' 'occupation_cat' 'basic_pay_x' 'basic_pay_squared_x'
 'age_15_19' 'age_20_24' 'age_25_29' 'age_30_34' 'age_35_39' 'age_40_44'
 'age_45_49' 'age_50_54' 'age_55_59' 'age_60_64' 'age_65_69' 'age_70_74'
 'age_75' 'age_UNSP_x' 'los_1' 'los_1_2' 'los_3_4' 'los_5_9' 'los_10_14'
 'los_15_19' 'los_20_24' 'los_25_29' 'los_30_34' 'los_35' 'los_UNSP_x'
 'occupation_fam_01_x' 'occupation_fam_02_x' 'occupation_fam_03_x'
 'occupation_fam_04_x' 'occupation_fam_05_x' 'occupation_fam_06_x'
 'occupation_fam_07_x' 'occupation_fam_08_x' 'occupation_fam_09_x'
 'occupation_fam_10_x' 'occupation_fam_11_x' 'occupation_fam_12_x'
 'occupation_fam_13_x' 'occupation_fam_14_x' 'occupation_fam_15_x'
 'occupation_fam_16_x' 'occupation_fam_17_x' 'occupation_fam_18_x'
 'occupation_fam_19_x' 'occupation_fam_20_x' 'occupation_fam_21_x'
 'superv_status_2_x' 'superv_status_4_x' 'superv_status_5_x'
 'superv_status_6_x' 'superv_status_7_x' 'superv_status_8_x'


,emp_id,agency_cat,occupation_cat,basic_pay_x,basic_pay_squared_x,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,...,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18
0,898655,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196
1,2257068,AA,P,161600.0,2.611456e+10,0,0,0,0,0,...,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196
2,1745142,AA,P,113346.0,1.284732e+10,0,0,0,1,0,...,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196
3,5050496,AA,P,75621.0,5.718536e+09,0,0,0,1,0,...,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196
4,8638776,AA,P,157100.0,2.468041e+10,0,0,0,0,0,...,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196,0.000196


# Calculate partial hazard for each employee

In [13]:
# Calculate partial hazards for each employee

from itertools import imap,starmap,izip
from operator import mul

# Function for sumproduct of normalized data and coefficients
def partial_hazard(norm_data,coefs):
    return np.exp(sum(map(mul,norm_data,coefs)))

pred_haz['partial_hazard'] = pred_haz.apply(lambda x: partial_hazard(x[5:59], x[60:114]), axis=1)
pred_haz['partial_hazard'][:5]

0    0.967377
1    0.955255
2    0.164793
3    0.547002
4    0.965528
Name: partial_hazard, dtype: float64

# Calculate Cumulative Hazard for each employee

In [15]:
# Create columns for cumulative hazard at time t for each employee
for i in time:                                             
  pred_haz['cum_haz_t' + str(i)] = pred_haz['partial_hazard'] * pred_haz['t' + str(i)]


print pred_haz.columns.values
pred_haz.tail()

['emp_id' 'agency_cat' 'occupation_cat' 'basic_pay_x' 'basic_pay_squared_x'
 'age_15_19' 'age_20_24' 'age_25_29' 'age_30_34' 'age_35_39' 'age_40_44'
 'age_45_49' 'age_50_54' 'age_55_59' 'age_60_64' 'age_65_69' 'age_70_74'
 'age_75' 'age_UNSP_x' 'los_1' 'los_1_2' 'los_3_4' 'los_5_9' 'los_10_14'
 'los_15_19' 'los_20_24' 'los_25_29' 'los_30_34' 'los_35' 'los_UNSP_x'
 'occupation_fam_01_x' 'occupation_fam_02_x' 'occupation_fam_03_x'
 'occupation_fam_04_x' 'occupation_fam_05_x' 'occupation_fam_06_x'
 'occupation_fam_07_x' 'occupation_fam_08_x' 'occupation_fam_09_x'
 'occupation_fam_10_x' 'occupation_fam_11_x' 'occupation_fam_12_x'
 'occupation_fam_13_x' 'occupation_fam_14_x' 'occupation_fam_15_x'
 'occupation_fam_16_x' 'occupation_fam_17_x' 'occupation_fam_18_x'
 'occupation_fam_19_x' 'occupation_fam_20_x' 'occupation_fam_21_x'
 'superv_status_2_x' 'superv_status_4_x' 'superv_status_5_x'
 'superv_status_6_x' 'superv_status_7_x' 'superv_status_8_x'
 'basic_pay_norm' 'basic_pay_squared_norm' 

,emp_id,agency_cat,occupation_cat,basic_pay_x,basic_pay_squared_x,age_15_19,age_20_24,age_25_29,age_30_34,age_35_39,...,cum_haz_t9,cum_haz_t10,cum_haz_t11,cum_haz_t12,cum_haz_t13,cum_haz_t14,cum_haz_t15,cum_haz_t16,cum_haz_t17,cum_haz_t18
769913,2797409,VA,P,309439.0,9.575249e+10,0,0,0,0,0,...,0.050281,0.056166,0.062585,0.066920,0.074368,0.079450,0.085245,0.099325,0.099325,0.099325
769914,8292536,VA,P,309439.0,9.575249e+10,0,0,0,0,0,...,0.163407,0.182532,0.203391,0.217481,0.241686,0.258199,0.277035,0.322792,0.322792,0.322792
769915,244892,VA,P,309695.0,9.591099e+10,0,0,0,0,0,...,0.367153,0.410124,0.456992,0.488649,0.543033,0.580137,0.622458,0.725267,0.725267,0.725267
769916,2263970,VA,P,277311.0,7.690139e+10,0,0,0,0,0,...,0.049211,0.054970,0.061252,0.065495,0.072784,0.077758,0.083430,0.097210,0.097210,0.097210
769917,2881482,VA,P,277695.0,7.711451e+10,0,0,0,0,0,...,0.344927,0.385296,0.429327,0.459068,0.510160,0.545017,0.584777,0.681362,0.681362,0.681362


In [32]:
# Prepare to write to csv
# Drop unnecessary cols (keep only calculated cumlative hazards)
# pred_haz_only = 
pred_haz_only = pred_haz[['emp_id', 'agency', 'partial_hazard',
                          'cum_haz_t1', 'cum_haz_t2', 'cum_haz_t3', 'cum_haz_t4', 
                          'cum_haz_t5', 'cum_haz_t6', 'cum_haz_t7', 'cum_haz_t8', 
                          'cum_haz_t9', 'cum_haz_t10', 'cum_haz_t11', 'cum_haz_t12', 
                          'cum_haz_t13', 'cum_haz_t14', 'cum_haz_t15', 'cum_haz_t16', 
                          'cum_haz_t17', 'cum_haz_t18']]
pred_haz_only.columns.values
pred_haz_only.head()

,emp_id,agency,partial_hazard,cum_haz_t1,cum_haz_t2,cum_haz_t3,cum_haz_t4,cum_haz_t5,cum_haz_t6,cum_haz_t7,...,cum_haz_t9,cum_haz_t10,cum_haz_t11,cum_haz_t12,cum_haz_t13,cum_haz_t14,cum_haz_t15,cum_haz_t16,cum_haz_t17,cum_haz_t18
0,898655,AA,0.967377,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
1,2257068,AA,0.955255,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187,...,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187,0.000187
2,1745142,AA,0.164793,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032
3,5050496,AA,0.547002,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107,...,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107,0.000107
4,8638776,AA,0.965528,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189


# Merge cumulative predicted hazard rates with Q2 2014 employees 
Using employee id and agency. This is to enable filtering in Tableau

In [20]:
%%storage read --object "gs://fedwork_bigquery_exports/Q2_2014_employees" --variable text

In [36]:
# Load data to dataframe
q2_2014_emps = pd.read_csv(StringIO(text)) 
print q2_2014_emps.shape
q2_2014_emps.columns.values

(1816452, 25)


array(['emp_id', 'file_dt', 'agency', 'duty_station', 'age',
       'education_level', 'pay_plan', 'grade', 'los', 'occupation',
       'occupation_cat', 'basic_pay', 'superv_status', 'type_appoint',
       'work_schedule', 'nsftp_ind', 'dod_ind', 'year', 'quarter',
       'acces_sep_ind', 'effective_dt', 'constructed', 'state_code',
       'agency_cat', 'occupation_fam'], dtype=object)

In [42]:
# Merge cumulative hazard to q2_2014_emps
q2_2014_emps_cum_haz = pd.merge(q2_2014_emps, pred_haz_only, how='left', left_on=['emp_id','agency_cat'], right_on=['emp_id', 'agency'])
print 'Shape of joined data frame: ', q2_2014_emps_cum_haz.shape
# there will be more employees in the Q2_2014_emps dataframe because it is not filtered by occupation categetory P and T

Shape of joined data frame:  (1816452, 45)


In [54]:
q2_2014_emps_cum_haz[q2_2014_emps_cum_haz['agency_cat'] == 'IN']["cum_haz_t12"].mean()

0.070078274870878948

In [46]:
# Write to file
q2_2014_emps_cum_haz.to_csv('pred_haz.csv', sep=',')